# Nanopore MLST Workflow

# 1. dorado Basecalling

#### Download Dorado basecaller
- if necessary

In [ ]:
!wget https://cdn.oxfordnanoportal.com/software/analysis/dorado-0.9.0-linux-x64.tar.gz -O /mnt/NanoporeRawData/dorado.tar.gz
!tar -xvf /mnt/NanoporeRawData/dorado.tar.gz -C /mnt/NanoporeRawData/

#### Fast5 Files Conversion
- if necessary

In [43]:
import os
import subprocess

# Define paths
raw_data_path = "/mnt/NanoporeRawData/2023-000029/"
fast5_path = os.path.join(raw_data_path, "fast5")
pod5_path = os.path.join(raw_data_path, "pod5")

# Function to check if the number of files match and none are empty
def check_pod5_files(fast5_path, pod5_path):
    fast5_files = [f for f in os.listdir(fast5_path) if f.endswith('.fast5')]
    pod5_files = [f for f in os.listdir(pod5_path) if f.endswith('.pod5')]

    # Check if counts match and ensure no empty files
    if len(fast5_files) != len(pod5_files):
        return False

    for pod5_file in pod5_files:
        if os.path.getsize(os.path.join(pod5_path, pod5_file)) == 0:  # Check if the file is empty
            return False

    return True

# Check if only the fast5 directory exists, and convert FAST5 to POD5
if os.path.exists(fast5_path) and not os.path.exists(pod5_path):
    print("FAST5 directory detected, converting to POD5...")

    # Install pod5 package if not already installed
    subprocess.run(["pip", "install", "pod5"], check=True)

    # Ensure the pod5 directory exists
    os.makedirs(pod5_path, exist_ok=True)

    # Convert FAST5 to POD5, using -o for output and -O for one-to-one mapping
    subprocess.run([
        "pod5", "convert", "fast5", 
        "-o", pod5_path,  # Output directory for POD5 files
        "-O", fast5_path,  # Parent directory for input files
        fast5_path  # Input path
    ], check=True)

# Rerun conversion if checks fail
if not check_pod5_files(fast5_path, pod5_path):
    print("File count mismatch or empty POD5 files detected. Re-running conversion...")
    subprocess.run([
        "pod5", "convert", "fast5", 
        "-o", pod5_path, 
        "-O", fast5_path,  
        fast5_path  
    ], check=True)

print("POD5 conversion completed.")

FAST5 directory detected, converting to POD5...


Converting 536 Fast5s: 100%|##########| 2142460/2142460 [4:05:15<00:00, 145.59Reads/s]  


POD5 conversion completed.


## 1-1. Pod5 Files Basecalling
- basecall
- convert BAM to fastq
- generate NanoPlot

In [6]:
import os
import subprocess
from packaging import version


''' ===== Configuration ===== '''
dorado_base_path = "/mnt/NanoporeRawData"
raw_data_path = "/mnt/NanoporeRawData/2024-000023/pod5"
basecall_data_path = "/mnt/Data/2024-000023/1_dorado"


''' ===== Workflow ===== '''
def get_latest_dorado_path(base_path):
    dorado_versions = [
        os.path.join(base_path, d) for d in os.listdir(base_path)
        if d.startswith("dorado-") and os.path.isdir(os.path.join(base_path, d))
    ]
    dorado_versions = sorted(
        dorado_versions, 
        key=lambda d: version.parse(d.split('-')[1].split('-')[0]),
        reverse=True
    )
    if dorado_versions:
        return os.path.join(dorado_versions[0], "bin", "dorado")
    else:
        raise FileNotFoundError("No Dorado versions found in the specified base path.")

try:
    dorado_path = get_latest_dorado_path(dorado_base_path)
    print(f"Using Dorado binary: {dorado_path}")
except FileNotFoundError as e:
    print(e)
    exit(1)

os.makedirs(basecall_data_path, exist_ok=True)

print("Running dorado basecaller...")
bam_file_path = os.path.join(basecall_data_path, "all.bam")
with open(bam_file_path, "w") as bam_file:
    subprocess.run([
        dorado_path, "basecaller", "sup", raw_data_path
    ], stdout=bam_file, check=True)

print("Converting BAM to FASTQ...")
subprocess.run([
    "bedtools", "bamtofastq", 
    "-i", bam_file_path,
    "-fq", os.path.join(basecall_data_path, "all.fastq")
], check=True)

print("Generating NanoPlot...")
subprocess.run([
    "NanoPlot", 
    "--fastq", os.path.join(basecall_data_path, "all.fastq"),
    "-o", basecall_data_path
], check=True)

print("Dorado basecalling and analysis completed.")


Using Dorado binary: /mnt/NanoporeRawData/dorado-0.9.0-linux-x64/bin/dorado
Running dorado basecaller...


[2024-12-30 10:45:26.903] [info] Running: "basecaller" "sup" "/mnt/NanoporeRawData/2024-000023/pod5"
[2024-12-30 10:45:32.723] [info]  - downloading dna_r10.4.1_e8.2_400bps_sup@v5.0.0 with httplib
[2024-12-30 10:46:36.456] [info] > Creating basecall pipeline
[2024-12-30 10:46:46.593] [info] Calculating optimized batch size for GPU "NVIDIA GeForce RTX 4090" and model /mnt/Data/2024-000023/.temp_dorado_model-c0e550a761c0f790/dna_r10.4.1_e8.2_400bps_sup@v5.0.0. Full benchmarking will run for this device, which may take some time.
[2024-12-30 10:47:03.368] [info] cuda:0 using chunk size 11520, batch size 128
[2024-12-30 10:47:03.856] [info] cuda:0 using chunk size 5760, batch size 128
[2024-12-30 13:45:46.330] [info] > Finished in (ms): 10721982
[2024-12-30 13:45:46.330] [info] > Simplex reads basecalled: 2688138
[2024-12-30 13:45:46.330] [info] > Simplex reads filtered: 29
[2024-12-30 13:45:46.330] [info] > Basecalled @ Samples/s: 2.841616e+06
[2024-12-30 13:45:47.924] [info] > Finished


Converting BAM to FASTQ...
Generating NanoPlot...
Dorado basecalling and analysis completed.


# 2. NanoACT Demultiplexing & Processing

## 2-1. Load NanoAct

In [7]:
import os

working_directory = os.getcwd()

# Change to home directory
os.chdir(os.path.expanduser("~"))

# Check if 'nanoACT' directory exists
if not os.path.exists("nanoACT"):
    # If not, clone the repository
    !git clone https://github.com/Raingel/nanoACT.git
    os.chdir(os.path.expanduser("~/nanoACT/"))
else:
    # If the directory exists, reset local changes and pull the latest updates
    os.chdir(os.path.expanduser("~/nanoACT/"))
    !git fetch --all > /dev/null 2>&1
    !git reset --hard origin/main > /dev/null 2>&1 # Force reset to the latest commit
    !git pull > /dev/null 2>&1

# Install requirements if necessary
"""
!pip install --upgrade pip
!pip install -r requirements.txt
"""

# Import nanoAct and initialize
from nanoact import nanoact
dumb = nanoact.NanoAct(TEMP = "/home/huanglabserver/nanoACT/temp/")

# Change back to the original working directory
os.chdir(working_directory)

# Verify the current working directory
print(os.getcwd())

/mnt/Data/2024-000023


## 2-2. Creating Barcoded_primers List
- Select target taxon
- Extract Sample ID and PCR ID
- Read Barcode file
- Read MLST primers file
- Rename PCR ID with real Sample ID
- Create Barcoded-primers file

In [3]:
import os
import csv


''' ===== Configuration ===== '''
project_id = "2024-000023"  # Set this to your project ID
target_taxon = "Candida"
replace_name = True # Set True to replace PCR ID with your own sample name according to reference.csv

''' File path settings '''
base_dir =  f"/mnt/Data/{project_id}"  # Base directory containing the csv files
barcode_ID_file = os.path.join(base_dir, f"{project_id}.csv")
reference_file = os.path.join(base_dir, "reference.csv")
MLST_primers_file = os.path.join(base_dir, "mlst_primers.csv")
output_file = os.path.join(base_dir, f"{project_id}_barcoded_MLST.csv")


''' ===== Workflow  ===== '''
barcode_IDs = []
MLST_primers = []
reference_data = {}

candida_genes = {"AAT1a", "ACC1", "ADP1", "MPIb", "SYA1", "VPS13", "ZWF1b"}
scedosporium_genes = {"ACT-1_4", "fRPB2-5f_7cR", "SOD2F3_R3", "Bt2a_2b", "1389_ITS4ngs", "Sau-CYP"}
trichophyton_genes = {"TrSQLE", "EF-Derm", "1389_ITS4ngs"}

if target_taxon.lower() == "candida":
    target_genes = candida_genes
elif target_taxon.lower() == "scedosporium":
    target_genes = scedosporium_genes
elif target_taxon.lower() == "trichophyton":
    target_genes = trichophyton_genes
else:
    raise ValueError(f"Unknown target_taxon: {target_taxon}")
    
if replace_name and os.path.exists(reference_file):
    with open(reference_file, 'r') as ref_file:
        reader = csv.DictReader(ref_file)  # Assuming CSV file
        for row in reader:
            sample = row['Sample'].split('_')[1]  # Extract part after '_'
            reference_data[row['PCR ID']] = sample  # Store mapping of PCR ID to sample
    print("Loaded reference data.")
else:
    print("reference.csv not found. Skipping sample replacement step.")

with open(barcode_ID_file, 'r') as barcode_file:
    reader = csv.DictReader(barcode_file)
    for row in reader:
        pcr_id = row['SampleID']  # Get PCR ID from barcode file
        if reference_data and pcr_id in reference_data:
            row['SampleID'] = reference_data[pcr_id]  # Replace SampleID with Sample from reference_data
        barcode_IDs.append(row)  # Add modified row to list

with open(MLST_primers_file, 'r') as primers_file:
    reader = csv.reader(primers_file)
    mlst_data = list(reader)
    MLST_primers_header = mlst_data[0]  # Extract the header
    MLST_primers = [row for row in mlst_data[1:] if row[0] in target_genes]  # Filter for target genes

output_data = [['SampleID', 'FwIndex'] + MLST_primers_header[2:]]  # New header

num_primers = len(MLST_primers)
for i, barcode_info in enumerate(barcode_IDs):
    barcode_ID = barcode_info['SampleID']
    fw_index = barcode_info['FwIndex']
    for j in range(num_primers):
        primer_index = (i * num_primers + j) % num_primers  # Cycling through primers for uniqueness
        primer_row = MLST_primers[primer_index]
        output_data.append([f'{barcode_ID}_{primer_row[0]}', fw_index] + primer_row[2:])

with open(output_file, 'w', newline='') as output_csv:
    writer = csv.writer(output_csv)
    writer.writerows(output_data)

print(f"File written to: {output_file}")


Loaded reference data.
File written to: /mnt/Data/2024-000023/2024-000023_barcoded_MLST.csv


## 2-3. Processing Sequences
- Quality filtering
- Double Demultiplexing
- Orientation 
- Trimming artificial reads
- Clustering using mmseqs2
- Consensus using MAFFT

In [ ]:
import os


''' ===== Configuration ===== '''
project_id = "2024-000023"  # Set this to your project ID


''' Analysis settings '''
input_format = "fastq"
output_format = "both" #輸出檔案的格式，預設為 'both'。可以是 fastq 或 fasta。'both' 代表同時輸出 fastq 和 fasta
mismatch_ratio_f = 0.1 #FwIndex容許的錯誤率，預設為0.15。例如barcode長度為20bp，則容許0.15*20=3bp的錯誤(edit distance)
mismatch_ratio_r = 0.1 #RvAnchor容許的錯誤率，預設為0.15


# Quality Filter Configuration
QSCORE = 9 #recommended 7-9
MIN_LEN = 400 #depends on the length of your reads
MAX_LEN = 850 #depends on the length of your reads

# Demultiplexing Configuration
expected_length_variation = 0.3 #預期的read長度變異，預設為0.3。例如預期的read長度為300bp，則容許0.3*300=90bp的變異
search_range = 150 #搜尋barcode的範圍，預設為150bp。代表搜尋範圍為前150bp和後150bp
rvc_rvanchor = True #預設為'False'。'True'則程式執行reverse-complement。

# Orientation Correction Configuration
orientation_search_range = 500 #搜尋FwPrimer和RvPrimer的範圍，預設為200bp。代表搜尋範圍為前200bp和後200bp。

# Trim Reads Configuration
fw_offset = 0 #從距離找到的切除位點開始往後切除幾個bp，預設為0，可以是負數。例如fw_offset=-10，則從距離找到的切除位點開始往前切除10個bp
rv_offset = 0 #從距離找到的切除位點開始往前切除幾個bp，預設為0，可以是負數。例如rv_offset=-10，則從距離找到的切除位點開始往後切除10個bp
discard_no_match = False
check_both_directions = True
reverse_complement_rv_col = True
trimming_search_range = 200

# Clustering Configuration (mmseqs_cluster)
cluster_min_seq_id = 0.98
cluster_mode = 0
cov_mode = 0
kmer_length = 15
kmer_per_seq = 20
sensitivity = 8.5
min_read_num = 3
suppress_output = True #suppress_output=False will output all details of the clustering process. Use it when unknown error occurs.

# Consensus Configuration (mafft_consensus)
minimal_reads = 2  # minimal_reads for consensus
max_reads = -1 #max_reads: 設定最多的序列數量，-1 代表不限制。例如max_reads=100，則只會隨機取100個序列進行排比
adjustdirection = False


''' ===== Workflow ===== '''
def process_data(project_id):
    data_base_path = f"/mnt/Data/{project_id}"
    src_path_dorado = os.path.join(data_base_path, "1_dorado/")
    des_path_nanofilt = os.path.join(data_base_path, "2_nanofilt/")
    des_path_demultiplex = os.path.join(data_base_path, "3_demultiplex/")
    des_path_orientation = os.path.join(data_base_path, "4_orientation/")  # Orientation output folder
    des_path_trimmed = os.path.join(data_base_path, "5_trimmed/")  # Trimming output folder
    des_path_mmseqs = os.path.join(data_base_path, "6_mmseqs/")  # Clustering output folder
    des_path_consensus = os.path.join(data_base_path, "7_consensus/")  # Consensus output folder
    barcode_index_file = os.path.join(data_base_path, f"{project_id}_barcoded_MLST.csv")


    # Step 1. Filter by Quality and Length
    filtered_fastq = dumb.qualityfilt(
        src = os.path.join(src_path_dorado, 'all.fastq'),
        des = des_path_nanofilt,
        name = 'all_qualityfilt.fastq',
        QSCORE = QSCORE,
        MIN_LEN = MIN_LEN,
        MAX_LEN = MAX_LEN
    )
    
    '''
    1. 使用 `fastq_reader` 函數處理原始定序檔案，每次讀取四行代表一個定序 read。
    2. 進行兩次解多工（demultiplexing）：
       1. 第一次基於 FwIndex 將 reads 分群。
       2. 第二次基於引子（primers）進行進一步的分群。
       3. 如果識別到唯一的 Sample ID，則將 read 分配到對應 Sample 的輸出檔案中。
       4. 如果條碼被截斷，則將 read 附加到 "TRUNCATED" 字典中。
       5. 如果識別出多個條碼，則將 read 附加到 "MULTIPLE" 字典中。
       6. 如果未識別條碼，則將 read 附加到 "UNKNOWN" 字典中。
       7. 最終根據條碼將已解多工的 reads 輸出到不同檔案。
    '''
    # Step 2. Demultiplexing
    demultiplexed = dumb.double_demultiplex(
        src = os.path.join(des_path_nanofilt, 'all_qualityfilt.fastq'),
        des = des_path_demultiplex,
        BARCODE_INDEX_FILE = barcode_index_file,
        primers = ['FwPrimer', 'RvPrimer'],
        mismatch_ratio_f = mismatch_ratio_f,
        mismatch_ratio_r = mismatch_ratio_r,
        expected_length_variation = expected_length_variation,
        search_range = search_range,
        rvc_rvanchor = rvc_rvanchor,
        input_format = input_format,
        output_format = output_format
    )
    
    # Step 3. Orientation correction
    orientation = dumb.orientation(
        src = des_path_demultiplex,
        des = des_path_orientation,
        input_format = input_format,
        output_format = output_format,
        BARCODE_INDEX_FILE = barcode_index_file,
        FwPrimer = "FwPrimer",
        RvPrimer = "RvPrimer",
        search_range = orientation_search_range
    )
    
    # Step 4. Trim Reads
    trimmed = dumb.trim_reads(
        src = des_path_orientation,
        des = des_path_trimmed,
        BARCODE_INDEX_FILE = barcode_index_file,
        fw_col = "FwPrimer",
        rv_col = "RvPrimer",
        input_format = input_format,
        output_format = output_format,
        mode = "table",
        fw_offset = fw_offset,
        rv_offset = rv_offset,
        mismatch_ratio_f = mismatch_ratio_f,
        mismatch_ratio_r = mismatch_ratio_r,
        discard_no_match = discard_no_match,
        check_both_directions = check_both_directions,
        reverse_complement_rv_col = reverse_complement_rv_col,
        search_range = trimming_search_range
    )

    # Step 5. mmseqs Clustering
    cluster = dumb.mmseqs_cluster(
        src = des_path_trimmed,
        des = des_path_mmseqs,
        min_seq_id = cluster_min_seq_id,
        cluster_mode = cluster_mode,
        cov_mode = cov_mode,
        k = kmer_length,
        kmer_per_seq = kmer_per_seq,
        s = sensitivity,
        min_read_num = min_read_num,
        input_format = input_format,
        output_format = output_format,
        suppress_output = suppress_output
    )

    # Step 6. Consensus
    consensus = dumb.mafft_consensus(
        src = des_path_mmseqs,
        des = des_path_consensus,
        minimal_reads = minimal_reads,
        input_format = "fas",
        max_reads = max_reads,
        adjustdirection = adjustdirection
    )

    return "Data processing complete."

# Call the function with the desired project ID:
result = process_data(project_id)
print(result)

# 3. Sequence Files Filtering

## 3-1. Check unique sequences
- Set exact_length = True for protein-coding genes

In [15]:
import os
import csv
import re
import shutil
from collections import defaultdict
from Bio import SeqIO

''' ===== Configuration ===== '''
project_id = "2024-000023"  # Set this to your project ID
exact_length = True  # Set to True to apply QC length filtering, False to skip this step
length_filtering_option = "!="  # Options: "<" or "!=", default is "!="

''' File path settings '''
base_dir = f"/mnt/Data/{project_id}"  # Base directory containing the csv files
input_directory = os.path.join(base_dir, "7_consensus")
output_directory = os.path.join(base_dir, "8_grouped_by_gene")
barcode_csv_path = os.path.join(base_dir, f"{project_id}_barcoded_MLST.csv")
output_csv_path = os.path.join(base_dir, "qc_step1.csv")  # Output CSV file path

''' ===== Workflow ===== '''
def load_gene_ids_from_barcode(barcode_csv_path):
    gene_ids_by_sample = defaultdict(lambda: {'gene_ids': [], 'Note': {}})
    with open(barcode_csv_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            sample_id, gene_id = row['SampleID'].split('_')
            gene_ids_by_sample[sample_id]['gene_ids'].append(gene_id)
            gene_ids_by_sample[sample_id]['Note'][gene_id] = int(row['Note'])
    print(f"Loaded gene IDs for {len(gene_ids_by_sample)} samples.")
    return gene_ids_by_sample

def process_sequences(input_directory, output_directory, gene_ids_by_sample, output_csv_path):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created output directory: {output_directory}")
    shutil.rmtree(output_directory)
    print(f"Cleared output directory: {output_directory}")
    
    pattern = re.compile(r'con_([A-Za-z0-9-]+)_([A-Za-z0-9]+)_cluster_(\d+)_r(\d+)\.(fas|fasta)')
    gene_summary = defaultdict(lambda: {
        'TotalSequences': 0,
        'UniqueSequences': 0,
        'LengthFilter': 0,
        'PassedQC': 0,
        'TotalReads': 0
    })
    sample_rows = []  # For storing rows per sample and gene ID
    for sample_id, data in gene_ids_by_sample.items():
        gene_ids = data['gene_ids']
        exact_lengths = data['Note']
        for gene_id in gene_ids:
            gene_dir = os.path.join(output_directory, gene_id)
            os.makedirs(gene_dir, exist_ok=True)
            
            unique_sequences = set()
            unique_sequences_pre_qc = set()
            sequences_checked, sequences_passed_qc, qc_read, total_reads = 0, 0, 0, 0
            
            for fasta_file in os.listdir(input_directory):
                match = pattern.match(fasta_file)
                if not match or match.group(1) != sample_id or match.group(2) != gene_id:
                    continue  # Skip files not matching current sample and gene ID
                    
                read_count = int(match.group(4))
                total_reads += read_count
                fasta_path = os.path.join(input_directory, fasta_file)
                sequences_to_write = []
                
                for record in SeqIO.parse(fasta_path, "fasta"):
                    sequences_checked += 1
                    sequence = str(record.seq)
                    if sequence not in unique_sequences_pre_qc:
                        unique_sequences_pre_qc.add(sequence)
                    if sequence not in unique_sequences:
                        if exact_length:
                            if length_filtering_option == "!=" and len(sequence) != exact_lengths[gene_id]:
                                continue
                            elif length_filtering_option == "<" and len(sequence) >= exact_lengths[gene_id]:
                                continue
                        unique_sequences.add(sequence)
                        sequences_to_write.append(record)
                        sequences_passed_qc += 1
                        qc_read += read_count  # Increment qc_read only for sequences passing QC
                        
                if sequences_to_write:
                    dest_path = os.path.join(gene_dir, fasta_file)
                    with open(dest_path, "w") as out_fasta:
                        SeqIO.write(sequences_to_write, out_fasta, "fasta")

            gene_summary[gene_id]['TotalSequences'] += sequences_checked
            gene_summary[gene_id]['UniqueSequences'] += len(unique_sequences_pre_qc)
            gene_summary[gene_id]['LengthFilter'] += len(unique_sequences)
            gene_summary[gene_id]['PassedQC'] += sequences_passed_qc
            gene_summary[gene_id]['TotalReads'] += total_reads

            if sequences_checked > 0:
                qc = f"'{sequences_passed_qc}/{sequences_checked}"
                passed_percentage = (sequences_passed_qc / sequences_checked) * 100
                read_passed = f"'{qc_read}/{total_reads}"
                read_percentage = (qc_read / total_reads) * 100
            else:
                qc = ""
                passed_percentage = ""
                read_passed = ""
                read_percentage = ""

            sample_rows.append({
                'SampleID': sample_id,
                'gene_id': gene_id,
                'qc': qc,
                'qc_passed_%': f"{passed_percentage:.2f}" if passed_percentage else "",
                'read_passed': read_passed,
                'read_passed_%': f"{read_percentage:.2f}" if read_percentage else ""
            })

    # Print summary report
    print("\nProcessing complete...")
    print("======== Summary report =======")
    for gene_id, stats in gene_summary.items():
        total_sequences = stats['TotalSequences']
        unique_sequences = stats['UniqueSequences']
        passed_qc = stats['PassedQC']
        total_reads = stats['TotalReads']
        qc_read = sum(int(re.sub(r"[^0-9]", "", row['read_passed'].split('/')[0])) for row in sample_rows if row['gene_id'] == gene_id and row['read_passed'])
        passed_percentage = (passed_qc / total_sequences) * 100 if total_sequences > 0 else 0
        read_percentage = (qc_read / total_reads) * 100 if total_reads > 0 else 0
         
        print(f"GeneID: {gene_id}")
        print(f"  Filtering Passed: {passed_qc}/{total_sequences} files ({passed_percentage:.2f}%)")
        print(f"  Unique Sequences: {unique_sequences}")
        print(f"  Reads Passed: {qc_read}/{total_reads} ({read_percentage:.2f}%)")
        print(f"  Files Copied: {passed_qc} file(s) to {output_directory}/{gene_id}.")
        print("-" * 32)

    # Write the sample-specific report to CSV
    with open(output_csv_path, mode='w', newline='') as csvfile:
        fieldnames = ['SampleID', 'gene_id', 'qc', 'qc_passed_%', 'read_passed', 'read_passed_%']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(sample_rows)
    print(f"\nCSV file written to {output_csv_path}.")

# Load gene IDs and exact length information
gene_ids_by_sample = load_gene_ids_from_barcode(barcode_csv_path)

# Run the modified function with the loaded gene IDs and exact length QC
process_sequences(input_directory, output_directory, gene_ids_by_sample, output_csv_path)

Loaded gene IDs for 35 samples.
Cleared output directory: /mnt/Data/2024-000023/8_grouped_by_gene

Processing complete...
======== Summary report =======
GeneID: AAT1a
  Filtering Passed: 786/3014 files (26.08%)
  Unique Sequences: 2838
  Reads Passed: 130901/407402 (32.13%)
  Files Copied: 786 file(s) to /mnt/Data/2024-000023/8_grouped_by_gene/AAT1a.
--------------------------------
GeneID: ACC1
  Filtering Passed: 30/54 files (55.56%)
  Unique Sequences: 51
  Reads Passed: 2184/2738 (79.77%)
  Files Copied: 30 file(s) to /mnt/Data/2024-000023/8_grouped_by_gene/ACC1.
--------------------------------
GeneID: ADP1
  Filtering Passed: 455/1857 files (24.50%)
  Unique Sequences: 1712
  Reads Passed: 117039/266442 (43.93%)
  Files Copied: 455 file(s) to /mnt/Data/2024-000023/8_grouped_by_gene/ADP1.
--------------------------------
GeneID: MPIb
  Filtering Passed: 571/2558 files (22.32%)
  Unique Sequences: 2422
  Reads Passed: 293973/523750 (56.13%)
  Files Copied: 571 file(s) to /mnt/Data

## 3-2. Translation Filtering
- Only for protein-coding genes

In [ ]:
'''latest'''

import os
import shutil
import re
import csv
from collections import defaultdict
from Bio import SeqIO
from Bio.Seq import Seq

# ===== Configuration =====
project_id = "2024-000023"  # Set this to your project ID

# File path settings
base_dir = f"/mnt/Data/{project_id}"
input_directory = os.path.join(base_dir, "8_grouped_by_gene")
output_directory = os.path.join(base_dir, "8_1_translation_filter")
qc_csv_path = os.path.join(base_dir, "qc_step1.csv")
output_csv_path = os.path.join(base_dir, "qc_step2.csv")

# ===== Workflow =====
def load_gene_ids_from_prev_qc(qc_csv_path):
    gene_data = {}
    gene_reads = defaultdict(lambda: defaultdict(int))
    gene_total_reads = defaultdict(int)
    warning_count = 0
    
    with open(qc_csv_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            sample_id = row['SampleID']
            gene_id = row['gene_id']
            qc = row['qc']
            reads = row['read_passed']
            total_files, total_reads = 0, 0
            
            if '/' in qc and '/' in reads:
                try:
                    total_files = int(qc.split('/')[1])
                    total_reads = int(reads.split('/')[1])
                    if int(qc.split('/')[0][1]) == 0 or int(reads.split('/')[0][1]) == 0:
                        print(f"Warning: No values in 'qc' and 'read_passed' columns for {sample_id} {gene_id}: '{qc}' and '{reads}'")
                        warning_count += 1
                except ValueError:
                    print(f"Warning: Invalid values in 'qc' and 'read_passed' columns for {sample_id} {gene_id}: '{qc}' and '{reads}'")
            else:
                print(f"Warning: No values in 'qc' and 'read_passed' columns for {sample_id} {gene_id}: '{qc}' and '{reads}'")
                warning_count += 1
            
            gene_data[(sample_id, gene_id)] = total_files
            gene_reads[sample_id][gene_id] = total_reads
            gene_total_reads[gene_id] += total_reads  # Add integer value for total_reads
    
    print(f"\nLoaded gene data for {len(gene_data)} samples.")
    print(f"Missing gene data: {warning_count}.\n")
    return gene_data, gene_reads, gene_total_reads



def translate_and_filter(input_directory, output_directory, gene_data, gene_reads, gene_total_reads):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created output directory: {output_directory}")
    shutil.rmtree(output_directory)
    print(f"Cleared output directory: {output_directory}")

    pattern = re.compile(r'con_([A-Za-z0-9-]+)_([A-Za-z0-9]+)_cluster_(\d+)_r(\d+)\.(fas|fasta)')
    gene_files = defaultdict(list)
    gene_total_files = defaultdict(int)
    gene_pass_count = defaultdict(int)

    for gene_id_dir in os.listdir(input_directory):
        gene_path = os.path.join(input_directory, gene_id_dir)
        if os.path.isdir(gene_path):
            for file in os.listdir(gene_path):
                if file.endswith(('.fas', '.fasta')) and file.startswith('con'):
                    match = pattern.match(file)
                    if match:
                        sample_id, gene_id, cluster_num, read_num, ext = match.groups()
                        read_count = int(read_num)
                        gene_files[(sample_id, gene_id)].append((os.path.join(gene_id_dir, file), read_count))
                    else:
                        print(f"File {file} in {gene_id_dir} does not match pattern.")

    def translate_sequence(seq):
        translations = []
        seq_rc = seq.reverse_complement()
        
        def trim_sequence(sequence):
            remainder = len(sequence) % 3
            if remainder != 0:
                sequence = sequence[:len(sequence) - remainder]
            return sequence
        
        for frame in range(3):
            forward_seq = trim_sequence(seq[frame:])
            translations.append(forward_seq.translate())
        for frame in range(3):
            reverse_seq = trim_sequence(seq_rc[frame:])
            translations.append(reverse_seq.translate())
        
        return translations

    csv_rows = []

    for (sample_id, gene_id), total_files in gene_data.items():
        files = gene_files.get((sample_id, gene_id), [])
        gene_total_files[gene_id] += total_files
        gene_read = gene_reads[sample_id].get(gene_id, 0)  # Use specific SampleID and gene_id pair
        passed_file_count, qc_read = 0, 0

        if files:
            gene_dir = os.path.join(output_directory, gene_id)
            if not os.path.exists(gene_dir):
                os.makedirs(gene_dir)

            for filepath, read_count in files:
                full_path = os.path.join(input_directory, filepath)
                with open(full_path, 'r') as handle:
                    for record in SeqIO.parse(handle, 'fasta'):
                        translations = translate_sequence(record.seq)
                        frames_without_stop_codons = [translation for translation in translations if '*' not in translation]
                        if frames_without_stop_codons:
                            qc_read += read_count
                            dest_file = os.path.join(gene_dir, os.path.basename(filepath))
                            shutil.copy(full_path, dest_file)
                            passed_file_count += 1

        gene_pass_count[gene_id] += passed_file_count
        if total_files > 0 and gene_read > 0:
            qc = f"'{passed_file_count}/{total_files}"
            passed_percentage = (passed_file_count / total_files) * 100
            read_passed = f"'{qc_read}/{gene_read}"
            read_percentage = (qc_read / gene_read) * 100
            csv_rows.append({
                'SampleID': sample_id,
                'gene_id': gene_id,
                'qc': qc,
                'qc_passed_%': f"{passed_percentage:.2f}",
                'read_passed': read_passed,
                'read_passed_%': f"{read_percentage:.2f}"
            })
        else:
            csv_rows.append({
                'SampleID': sample_id,
                'gene_id': gene_id,
                'qc': '',
                'qc_passed_%': '',
                'read_passed': '',
                'read_passed_%': ''
            })
    
    print("\nProcessing complete...")
    print("======== Summary report =======")
    for gene_id in gene_pass_count:
        total_files = gene_total_files[gene_id]
        passed_file_count = gene_pass_count[gene_id]
        qc_read = sum(int(re.sub(r"[^0-9]", "", row['read_passed'].split('/')[0])) for row in csv_rows if row['gene_id'] == gene_id and row['read_passed'])
        total_reads = sum(gene_reads[sample_id][gene_id] for sample_id in gene_reads if gene_id in gene_reads[sample_id])
        read_percentage = (qc_read / total_files) * 100 if total_files > 0 else 0
        print(f"GeneID: {gene_id}")
        print(f"  Filtering Passed: {passed_file_count}/{total_files} files ({(passed_file_count / total_files) * 100:.2f}%)")
        print(f"  Reads Passed: {qc_read}/{total_reads} ({read_percentage:.2f}%)")
        print(f"  Files Copied: {passed_file_count} file(s) to {output_directory}/{gene_id}.")
        print("-" * 32)

    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['SampleID', 'gene_id', 'qc', 'qc_passed_%', 'read_passed', 'read_passed_%']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(csv_rows)
    print(f"\nCSV file written to {output_csv_path}.")

# Load gene_id information from the barcode file
gene_data, gene_reads, gene_total_reads = load_gene_ids_from_prev_qc(qc_csv_path)

# Run the modified function with the loaded gene data
translate_and_filter(input_directory, output_directory, gene_data, gene_reads, gene_total_reads)


Loaded gene data for 245 samples.
Missing gene data: 39.

Cleared output directory: /mnt/Data/2024-000023/8_1_translation_filter


In [29]:
reads = "'0/4"
read = reads.split('/')[0][1]
print(read)

0


## 3-3. Extract Top Read Files

In [17]:
'''latest'''

import os
import shutil
import re
import csv
from collections import defaultdict

# ===== Configuration =====
project_id = "2024-000023"  # Set this to your project ID
top_files_no = 4  # Set the top numbers that you wish to extract

# File path settings
base_dir = f"/mnt/Data/{project_id}"  # Base directory containing the csv files
input_directory = os.path.join(base_dir, "8_1_translation_filter")
output_directory = os.path.join(base_dir, "9_final_result")
qc_csv_path = os.path.join(base_dir, "qc_step2.csv")
output_csv_path = os.path.join(base_dir, f"{project_id}_sequencing_report.csv")

# ===== Workflow =====
def load_gene_ids_from_prev_qc(qc_csv_path):
    gene_data = {}
    gene_reads = defaultdict(lambda: defaultdict(int))
    gene_total_reads = defaultdict(int)
    warning_count = 0
    
    with open(qc_csv_path, mode='r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            sample_id = row['SampleID']
            gene_id = row['gene_id']
            qc = row['qc']
            reads = row['read_passed']
            total_files, total_reads = 0, 0
            
            if '/' in qc and '/' in reads:
                try:
                    total_files = int(qc.split('/')[1])
                    total_reads = int(reads.split('/')[1])
                except ValueError:
                    print(f"Warning: Invalid values in 'qc' and 'read_passed' columns for {sample_id} {gene_id}: '{qc}' and '{reads}'")
            else:
                print(f"Warning: No values in 'qc' and 'read_passed' columns for {sample_id} {gene_id}: '{qc}' and '{reads}'")
                warning_count += 1
            
            gene_data[(sample_id, gene_id)] = total_files
            gene_reads[sample_id][gene_id] = total_reads
            gene_total_reads[gene_id] += total_reads  # Add integer value for total_reads
    
    print(f"\nLoaded gene data for {len(gene_data)} samples.")
    print(f"Missing gene data: {warning_count}.\n")
    return gene_data, gene_reads, gene_total_reads

def extract_top_reads_files(input_directory, output_directory, gene_data, gene_reads, gene_total_reads):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created output directory: {output_directory}")
    shutil.rmtree(output_directory)
    print(f"Cleared output directory: {output_directory}")

    pattern = re.compile(r'con_([A-Za-z0-9-]+)_([A-Za-z0-9]+)_cluster_(\d+)_r(\d+)\.(fas|fasta)')
    gene_files = defaultdict(list)
    gene_total_files = defaultdict(int)
    gene_pass_count = defaultdict(int)

    for gene_id_dir in os.listdir(input_directory):
        gene_path = os.path.join(input_directory, gene_id_dir)
        if os.path.isdir(gene_path):
            for file in os.listdir(gene_path):
                if file.endswith(('.fas', '.fasta')) and file.startswith('con'):
                    match = pattern.match(file)
                    if match:
                        sample_id, gene_id, cluster_num, read_num, ext = match.groups()
                        read_count = int(read_num)
                        gene_files[(sample_id, gene_id)].append((os.path.join(gene_id_dir, file), read_count))
                    else:
                        print(f"File {file} in {gene_id_dir} does not match pattern.")

    csv_rows = []
    for (sample_id, gene_id), total_files in gene_data.items():
        files = gene_files.get((sample_id, gene_id), [])
        gene_total_files[gene_id] += total_files
        gene_read = gene_reads[sample_id][gene_id]  # Use specific SampleID and gene_id pair
        passed_file_count, qc_read = 0, 0
        
        if files:
            gene_dir = os.path.join(output_directory, gene_id)
            if not os.path.exists(gene_dir):
                os.makedirs(gene_dir)
                
        if gene_files[(sample_id, gene_id)]:
            files = sorted(gene_files[(sample_id, gene_id)], key=lambda x: x[1], reverse=True)[:top_files_no]
            for file, read_count in files:
                src_path = os.path.join(input_directory, file)
                dest_path = os.path.join(gene_dir, os.path.basename(file))
                if os.path.exists(src_path):
                    shutil.copy(src_path, dest_path)
                    passed_file_count += 1
                    qc_read += read_count

        gene_pass_count[gene_id] += passed_file_count
        if total_files > 0 and gene_read > 0:
            qc = f"'{passed_file_count}/{total_files}"
            passed_percentage = (passed_file_count / total_files) * 100
            read_passed = f"'{qc_read}/{gene_read}"
            read_percentage = (qc_read / gene_read) * 100
            if qc_read == int(0):
                csv_rows.append({
                    'SampleID': sample_id,
                    'gene_id': gene_id,
                    'qc': '',
                    'qc_passed_%': '',
                    'read_passed': '',
                    'read_passed_%': ''
                })
            else:
                csv_rows.append({
                    'SampleID': sample_id,
                    'gene_id': gene_id,
                    'qc': qc,
                    'qc_passed_%': f"{passed_percentage:.2f}",
                    'read_passed': read_passed,
                    'read_passed_%': f"{read_percentage:.2f}"
                })
        else:
            csv_rows.append({
                'SampleID': sample_id,
                'gene_id': gene_id,
                'qc': '',
                'qc_passed_%': '',
                'read_passed': '',
                'read_passed_%': ''
            })
            
    # Summary report for each gene_id
    print("\nProcessing complete...")
    print("======== Summary Report ========")
    for gene_id in gene_pass_count:
        total_files = gene_total_files[gene_id]
        passed_file_count = gene_pass_count[gene_id]
        qc_read = sum(int(re.sub(r"[^0-9]", "", row['read_passed'].split('/')[0])) for row in csv_rows if row['gene_id'] == gene_id and row['read_passed'])
        total_reads = gene_total_reads[gene_id]
        passed_percentage = (passed_file_count / total_files) * 100 if total_files > 0 else 0
        read_percentage = (qc_read / total_reads) * 100 if total_reads > 0 else 0
        print(f"GeneID: {gene_id}")
        print(f"  Filtering Passed: {passed_file_count}/{total_files} files ({passed_percentage:.2f}%)")
        print(f"  Reads Passed: {qc_read}/{total_reads} ({read_percentage:.2f}%)")
        print(f"  Files Copied: {passed_file_count} file(s) to {output_directory}/{gene_id}.")
        print("-" * 32)

    # Write the sorted rows to the CSV with the simplified structure
    with open(output_csv_path, mode='w', newline='') as csv_file:
        fieldnames = ['SampleID', 'gene_id', 'qc', 'qc_passed_%', 'read_passed', 'read_passed_%']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(csv_rows)
    print(f"\nCSV file written to {output_csv_path}.")

# Load gene_id information from qc_step2.csv
gene_data, gene_reads, gene_total_reads = load_gene_ids_from_prev_qc(qc_csv_path)

# Run the function with gene_id filtering and QC check
extract_top_reads_files(input_directory, output_directory, gene_data, gene_reads, gene_total_reads)


Loaded gene data for 245 samples.
Created output directory: /mnt/Data/2024-000023/9_final_result
Cleared output directory: /mnt/Data/2024-000023/9_final_result

Processing complete...
======== Summary Report ========
GeneID: AAT1a
  Filtering Passed: 93/3014 files (3.09%)
  Reads Passed: 120899/407402 (29.68%)
  Files Copied: 93 file(s) to /mnt/Data/2024-000023/9_final_result/AAT1a.
--------------------------------
GeneID: ACC1
  Filtering Passed: 30/54 files (55.56%)
  Reads Passed: 2184/2738 (79.77%)
  Files Copied: 30 file(s) to /mnt/Data/2024-000023/9_final_result/ACC1.
--------------------------------
GeneID: ADP1
  Filtering Passed: 104/1857 files (5.60%)
  Reads Passed: 110951/266442 (41.64%)
  Files Copied: 104 file(s) to /mnt/Data/2024-000023/9_final_result/ADP1.
--------------------------------
GeneID: MPIb
  Filtering Passed: 108/2558 files (4.22%)
  Reads Passed: 287662/523750 (54.92%)
  Files Copied: 108 file(s) to /mnt/Data/2024-000023/9_final_result/MPIb.
--------------